In [10]:
# Step 1: Data Preprocessing
import pandas as pd
from tqdm import tqdm

# Read the xlsx file into a DataFrame
# Data from:
# https://nsidc.org/arcticseaicenews/sea-ice-tools/
# All daily (single day and five-day trailing average) extent values in one file, updated daily (Sea_Ice_Index_Daily_Extent_G02135_v3.0.xlsx)
file_path = 'Sea_Ice_Index_Daily_Extent_G02135_v3.0.xlsx'
df = pd.read_excel(file_path, engine='openpyxl', header=None)

# Extract the year values from the first column after removing non-numeric entries
years = df.iloc[0, 2:-3].dropna().astype(int).values

# Forward-fill missing month names with the previous non-NaN month value
df[0].fillna(method='ffill', inplace=True)

# make sure it has the right number of rows (1 + 366 = 367)
df = df.iloc[:367]

# Drop the first row (year header) and reset the index
df = df.iloc[1:].reset_index(drop=True)

# Drop the last three columns and reset the index
df = df.iloc[:, 0:-3].reset_index(drop=True)

# Preprocess the data for prophet package (ds and y columns)

# Initialize variables to hold month and day information
month = ''
day = ''

# Create a new DataFrame with the 'ds' and 'y' columns
df_new = pd.DataFrame(columns=['ds', 'y'])

df_new_row_index = 0

# Iterate through, column by column (skipping the first two)
for col_index in range(2, df.shape[1]):
    year = years[col_index - 2]
    print(f"Pre-processing data for year {year} ...")
    
    # Iterate through the DataFrame rows
    for index, row in df.iterrows():
        # If the row has a non-NaN value in the first column, it contains the month name
        if pd.notna(row[0]):
            month = row[0]
            day = int(row[1])
            
        # Check if the date falls on February 29 and if it is a non-leap year
        leap_year = (year % 4 == 0)
        if month == 'February' and day == 29 and not leap_year:
            # Skips this date since it is not a real date and there is no data
            continue

        # Construct the date string by combining month, day, and year information
        year_str = str(int(year))
        day_str = str(int(day))
        date_str = f"{month} {day_str}, {year_str}"

        # Add the constructed date strings as the 'ds' column in the DataFrame
        new_ds_value = pd.to_datetime(date_str, format='%B %d, %Y', errors='coerce')
        new_y_value = row[col_index]
        
        # Append the new 'ds' and 'y' values to the DataFrame
        df_new.loc[df_new_row_index, 'ds'] = new_ds_value
        df_new.loc[df_new_row_index, 'y'] = new_y_value
        df_new_row_index += 1
    
# Set any missing or empty values to NaN
df_new.replace(r'^\s*$', pd.NA, regex=True, inplace=True)

Pre-processing data for year 1978 ...
Pre-processing data for year 1979 ...
Pre-processing data for year 1980 ...
Pre-processing data for year 1981 ...
Pre-processing data for year 1982 ...
Pre-processing data for year 1983 ...
Pre-processing data for year 1984 ...
Pre-processing data for year 1985 ...
Pre-processing data for year 1986 ...
Pre-processing data for year 1987 ...
Pre-processing data for year 1988 ...
Pre-processing data for year 1989 ...
Pre-processing data for year 1990 ...
Pre-processing data for year 1991 ...
Pre-processing data for year 1992 ...
Pre-processing data for year 1993 ...
Pre-processing data for year 1994 ...
Pre-processing data for year 1995 ...
Pre-processing data for year 1996 ...
Pre-processing data for year 1997 ...
Pre-processing data for year 1998 ...
Pre-processing data for year 1999 ...
Pre-processing data for year 2000 ...
Pre-processing data for year 2001 ...
Pre-processing data for year 2002 ...
Pre-processing data for year 2003 ...
Pre-processi

In [21]:
# Remove empty values from future dates at end of the frame
# Find the index of the last non-empty 'y' value starting from the last row
last_non_empty_y_index = None
for i in range(len(df_new) - 1, -1, -1):
    if pd.notna(df_new.loc[i, 'y']):
        last_non_empty_y_index = i
        break

# If no non-empty 'y' value is found, the DataFrame will be empty
if last_non_empty_y_index is not None:
    # Slice the DataFrame to keep rows from last_non_empty_y_index onwards
    df_new = df_new.loc[0:last_non_empty_y_index]

print('Done pre-processing data:')
print(df_new)

# Fill missing values with interpolation (optional)
#df.interpolate(method='linear', axis=0, inplace=True)

Done pre-processing data:
              ds      y
0     1978-01-01    NaN
1     1978-01-02    NaN
2     1978-01-03    NaN
3     1978-01-04    NaN
4     1978-01-05    NaN
...          ...    ...
16630 2023-07-14  8.375
16631 2023-07-15  8.286
16632 2023-07-16  8.143
16633 2023-07-17  8.054
16634 2023-07-18  8.041

[16635 rows x 2 columns]


In [36]:
# Step 2: Install and Import Required Libraries
# You might need to install fbprophet using "pip install fbprophet" first.
from prophet import Prophet

# Step 3: Forecasting with Prophet
# Create a Prophet model
model = Prophet(interval_width=0.995)

# Rename the 'ds' and 'y' columns as required by Prophet
df.rename(columns={years[0]: 'y'}, inplace=True)

# Fit the model to your data
model.fit(df_new)

# Create a DataFrame with future dates for prediction
future_dates = model.make_future_dataframe(periods=365, include_history=False)  # Forecasting 365 days into the future

# Make predictions for the future dates
forecast = model.predict(future_dates)

# Print the forecasted values
print("Forecast DataFrame:")
print(forecast)

11:56:14 - cmdstanpy - INFO - Chain [1] start processing
11:56:29 - cmdstanpy - INFO - Chain [1] done processing


Forecast DataFrame:
            ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0   2023-07-19  10.491813    6.950365    8.996493    10.491813    10.491813   
1   2023-07-20  10.491873    6.800764    8.828697    10.491873    10.491873   
2   2023-07-21  10.491933    6.690147    8.801882    10.491933    10.492310   
3   2023-07-22  10.491993    6.645716    8.770383    10.491993    10.493066   
4   2023-07-23  10.492052    6.482981    8.615168    10.492052    10.493489   
..         ...        ...         ...         ...          ...          ...   
360 2024-07-13  10.513349    7.268967    9.664764     9.917808    11.141690   
361 2024-07-14  10.513409    7.153028    9.464771     9.915936    11.143518   
362 2024-07-15  10.513469    7.133294    9.332966     9.914063    11.145346   
363 2024-07-16  10.513528    7.109833    9.178645     9.912191    11.147174   
364 2024-07-17  10.513588    6.908238    9.202663     9.910319    11.149002   

     additive_terms  additive_t

In [37]:
import pandas as pd
import plotly.graph_objects as go
from prophet.plot import plot_plotly, plot_components_plotly

#pip install ipywidgets

# Assuming you have the forecast data in a DataFrame called forecast_df
forecast_df = forecast.loc[-365:]
# The DataFrame should contain a 'ds' column with the dates and a 'yhat' column with the forecasted values.

# Convert the 'ds' column to pandas datetime objects (if not already)
forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])

# Plot the forecast using plot_plotly
fig = plot_plotly(model, forecast_df)
# Set the default range for the x-axis (datetimestamp)
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2024-12-31')
fig.update_xaxes(range=[start_date, end_date])
fig.update_layout(title_text="Arctic Sea Ice-Extent Forecast only using historical data - 99.5% CI)")
fig.update_yaxes(title_text="Extent (10^6 km^2)")
fig.show()

# Plot the forecast components using plot_components_plotly
# fig_comp = plot_components_plotly(model, forecast_df)
# fig_comp.show()

In [38]:
forecasted_values = forecast['yhat']
forecasted_lower_bounds = forecast['yhat_lower']
forecasted_upper_bounds = forecast['yhat_upper']

# Print the minimum value for the forecast
min_forecast = forecasted_values.min()
print("Minimum Forecast Value:", round(min_forecast, 3))

# Print the ± range values for the forecast
print("99.5% Interval:")
print("Lower Bound:", round(forecasted_lower_bounds.min(),3))
print("Upper Bound:", round(forecasted_upper_bounds.min(),3))


Minimum Forecast Value: 4.943
99.5% Interval:
Lower Bound: 3.715
Upper Bound: 5.961
